In [1]:
import os
import re
import numpy as np

%matplotlib ipympl
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (9, 7)

from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

# Imports, code

In [2]:
import lsst.daf.persistence as dafPersist
from pfs.datamodel.pfsConfig import TargetType
from pfs.drp.stella.utils.pfiFocus import estimateFiberFluxes, plotVariantOffsets, plotRadialProfiles, plotRms, rmsFromCenteredFibers

if False:
    from pfs.drp.stella.utils import DataId
else:
    DataId = dict

ModuleNotFoundError: No module named 'pfs.drp.stella.utils.pfiFocus'

# Butler

In [ ]:
def butlerName(_butler=None):
    if _butler is None:
        _butler = butler
    root = _butler._repos.inputs()[0]._repoArgs.root
    root = re.sub(r"^.*/rerun/", "", root)
    
    return root

butlers = {}

repo = "/work/drp" 

reruns = [
    "rhl/tmp",
    "rhl/focus",
]
for rerun in reruns:
    kwargs = {}
    if rerun.startswith('/'):
        dataDir = rerun
    else:
        dataDir = os.path.join(repo, "rerun", rerun)

    if rerun in ["rhl/focus"]:
        kwargs.update(calibRoot="/work/drp/CALIB-kiyoyabe-20240311")

    if not os.path.exists(dataDir):
        continue

    butlers[rerun] = dafPersist.Butler(dataDir, **kwargs)

if os.path.exists("/work/drp"):
    kwargs = {}
    repoRoot = "/work/drp"

    calibName = "CALIB-20220630"
    kwargs.update(calibRoot=os.path.join(repoRoot, calibName))
    rerun = f'drpActor/{calibName}'

    butlers["drp"] = dafPersist.Butler(os.path.join(repoRoot, 'rerun', rerun), **kwargs)

butler = butlers[list(butlers)[0]]   # default

# Process data

In [ ]:
if False:
    visits = sorted([108315] + list(range(108317, 108339+1)))
else:
    visits = sorted([108507] + list(range(108509, 108525 + 1))) # 

filterName = "g_gaia"
pfsConfig = butler.get("pfsConfig", visit=visits[1]).select(targetType=~TargetType.ENGINEERING)
nJy = pfsConfig.getPhotometry(filterName)
mag = 8.90 - 2.5*np.log10(nJy*1e-9)

## Distribution of fluxes

## Read data

In [ ]:
fig = 'tmp'; plt.close(fig); fig = plt.figure(fig)

plt.hist(mag, bins=21)
plt.xlabel(f"{filterName}")
plt.ylabel("N")
plt.title(f"{pfsConfig.designName}");

In [ ]:
if False:
    from importlib import reload
    import pfs.drp.stella.utils.pfiFocus
    estimateFiberFluxes = reload(pfs.drp.stella.utils.pfiFocus).estimateFiberFluxes

In [ ]:
try:
    cache
except NameError:
    cache = {}

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('error', UserWarning)
    pass

cache = estimateFiberFluxes(butler, visits, windows=["b", "r", "b1", "b2", "r1", "r2"],
                            missingCamera=lambda arm, s: (arm == 'b' and s in [3]) or (arm == 'n' and s in [1, 4]),
                            cache=cache)

## Visualize offsets

In [ ]:
fig = 1; plt.close(fig); fig = plt.figure(fig)

plotVariantOffsets(cache, title=f"{pfsConfig.designName}", figure=fig)

## Make radial profile plots

In [ ]:
fig = 2; plt.close(fig); fig = plt.figure(fig)

magMin = 15
magMax = 17

windows=["b", "r"] if False else ['b1', 'b2', 'r1', 'r2']


with np.testing.suppress_warnings() as suppress:
    suppress.filter(RuntimeWarning, "RuntimeWarning: Mean of empty slice")

    rms, meanCenteredFlux = plotRadialProfiles(windows=windows,
                                           nbin=10, rmax=200, mag=mag, filterName=filterName, magMin=magMin, magMax=magMax, cache=cache,
                                           normPercentile=100, title="", figure=fig)

## RMS as a function of focus

In [ ]:
fig = 3; plt.close(fig); fig = plt.figure(fig)

plotRms(rms, byQuadrant=True, title="", cache=cache, figure=fig)

## Focus sweep using immobile cobras, unaffected by variant

In [ ]:
fig = 4; plt.close(fig); fig = plt.figure(fig)

#windows = ['b1', 'b2', 'r1', 'r2']
rmsFromCenteredFibers(meanCenteredFlux, byQuadrant=True, cache=cache, title="", figure=fig)